# Data scientist workflow

Let's begin a full data scientist workflow on a simplified use case. 

**Situation**: Today Thermal Models (also called TMM: Thermal Mathematical Models) are very (very) slow - order of magnitude: 1 hours per simulation. For a complete thermal analysis, thousand of simulations have to be launch, to test the various space environment (test the different manoevers, on various date of the year, with various equipement ON/OFF...). 

Your purpose is to propose a Machine learning model permitting to *emulate* the TMM with a drastically reduced computing time.

**Chalenge**:
1. Find a Machine learning model which *emulate* the best TMM behaviour. More precisely, you have to propose an ML model with error < 1°.
2. Your model shall be run a simulation on a reasonable computing time: < 1 second for one orbit simulated.

**TMM used**: You will use a simplified model. This model is composed by a satellite with 45 thermal nodes (45 temperatures are outputed by the model). 

![img/train_sat.png](img/train_sat.png)
*Illustration: the satellite used. Left: external model, right internal model.*

* 9 TMM Input: 

    * 6 dissipation parameters: in [0.1, 2]
        * QI_BATTERY: dissipation of the battery 
        * QI EPC: dissipation of the EPC: Electronic Power Converter
        * QI_OMUX: dissipation of the OMUX: Output Multiplexer
        * QI_PCDU: dissipation of the PCDU (Power Conditioning and Distribution Unit)
        * QI_TRANSPONDER: dissipation of the transponder
        * QI_TWT: Dissipation of the Travelling Wave Tube (repeater)
    * 3 coupling parameters: in [0.1 ; 5]
        * Wall_MLI: Coefiscient of heat insulation of the MLI (Multi Layer Insulation)
        * Wall_Units: Coeffiscient of heat transfert between the equipements and the wall  
        * Wall_Wall: : Coeffiscient of rejection of heat between the wall and space (measure efficiency of the heat pipes).


* TMM Output: 45 Temepratures (one per node).

**Work**:
1. Generate data
2. Pre-process data (train test split, ...)
3. Choose a reference model and a loss function
4. Choose and train your ML model
5. Evaluate models (scores, comparison to reference, robustness, ...)
6. [facultative] Optimize the hyper-parameters of your models

**Import usefull libs here**

In [ ]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from time import time

# Usefull
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Linear model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# XGBoost
from xgboost import XGBRegressor

# Gaussian process
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF, Matern, RationalQuadratic, ExpSineSquared, ConstantKernel

# Other usefull models
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVR

# Hyper-parameters tunning
from sklearn.model_selection import RandomizedSearchCV
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# 0/ Generate data

**General confs**

In [ ]:
filename_input = "data/input_df.pickle"
filename_outputs = "data/target_df.pickle"

**Read data**

In [ ]:
# Read filename containing the inputs...
with open(filename_input, 'rb') as f:
    df_in = pickle.load(f)
    
# ... and the outputs
with open(filename_outputs, 'rb') as f:
    df_out = pickle.load(f)

    
df_in.head()

In [ ]:
df_out.head()

**Pre-process data: Clean**

In [ ]:
# Drop the last column (contains the node "space", always at -269°C)
df_out = df_out.drop(columns=44)
df_out.head()

**Pre-process data: Transform into Numpy DataFrame**

More convenient for data manipulation

In [ ]:
np_in, np_out = df_in.values, df_out.values

# 1/ Exercice 1: Pre-process data

**Purpose of the exercice:**: Find in autonomy the better way to pre-process data.

**Instructions**:
1. Split data into *train* and *test* sample. You can use the sklearn `train_test_split` to do that. (just search on Google)
2. *Scale* your data using the sklearn `StandardScaler` (just google it)
3. Analyse the data: plot values of T° for node 14 before and after scaling.

*Important note about scaling*: 
A scaler works in two steps: 
1. `fit`: compute statistics on the distribution (e.g. the average and std values of the dataset for `StandardScaler`)
2. `transform`: effectively apply the operation of scaling (e.g. substract by average value and divide by std).

The operation 1. (`fit`) must be performed on **Train** data only to get sure no information of the **validation** sample is used.

### 1.1 Train test split to avoid overtraining

NB: `train_test_split` also shuffle data.

### 1.2 Scaling

We choose to scale the data. It is a process often used to "help" the models prediction (better scores when trained on scaled data). 

In this BE, we choose to use `StandardScaler`. See [this post](https://stackoverflow.com/questions/40758562/can-anyone-explain-me-standardscaler) for explanations.

Note that the scaler is fitted on **train** data only. We are sure that no information is transmitted between train and test.

To apply the unscaling operation, just apply `scaler_out.inverse_transform(y_data_to_unscale)`.

### 1.3. Analyze your data

Let's visualize the scaling effect.

In [ ]:
# Visualize the scaling effect on node:
node_id = 14

As you can see, scaled data ranges between 0 and 1. 

**Usefull function**

Here is a usefull function to analyse your data. It permits to show the distribution of values for each node.

In [ ]:
plt.figure(figsize = (15, 5))

plt.boxplot(y_train)
plt.title("Ditribution of T°")
plt.xlabel("Node id")
plt.ylabel("T°")
plt.show()

# 2/ Choose the cost function

Choose the criterion of evaluation of your model. 

In [ ]:
# List of criterions:


def compute_quality_criterions(y_pred, y_ref):
    """
    Compute all the criterion of quality of your prediction. 
    Permits to efficiently compare models quality regarding multiple criterions.
    
    :return: dict containing:
        * Key: name of the criterion
        * Value: Value of the criterion
    """
    return {"mae": mean_absolute_error(y_pred, y_ref), # mean absolute error
            "mse": mean_squared_error(y_pred, y_ref),  # mean squared error
            "std": np.std(y_pred - y_ref).mean(),  # standard deviation 
            "max_absolute_error": np.max(np.max(np.abs(y_pred - y_ref))),  # maximal absolute error 
           }

def print_quality_criterion(y_pred, y_ref):
    """
    Display the quality criterions.
    """    
    d = compute_quality_criterions(y_pred, y_ref)
       
    for k, v in d.items():
        # ex: mae: 0.1
        print(f"{k}: {v:.1f}")  # .1f: 1 digit after coma       



Do not forget to unscale your data BEFORE applying the criterion (else, no sense).

# 3/ Reference model

Choose a naive model permitting to have a reference.

In this section, I show you an example of code to reproduce. The only element to modify is the model choosen (line `model = ...`).

### 3.1. Linear regression

In [ ]:
# Define the model to use
model = LinearRegression()

# Train the model
model.fit(X_train_scaled, y_train_scaled)

### 3.2. Evaluate quality of the *reference* model

In [ ]:
# Evaluate the model on the validation sample
y_pred_scaled = model.predict(X_test_scaled)

# Un-scale the prediction (remember, X and y are scaled !)
y_pred = scaler_out.inverse_transform(y_pred_scaled)

In [ ]:
# Display the scores on this model
print_quality_criterion(y_pred, y_test)

# Plot the dispertion of errors for each node.
plt.figure(figsize=(20,5))

plt.title("Difference prediction - reference on TEST sample")
plt.xlabel("Node id")
plt.ylabel("T°")
plt.boxplot(y_pred - y_test)
plt.show()

**Bilan**: Ok, the linear regression model is very bad. 

But we have our reference. Let's try now to do a better job.

# 4/ Exercice 2: Benchmark ML models


**Purpose of the exercice:** This is the main exercice. Purpose is to test multiple machine learning models and compare them. Through this exercice, I want to show you a realistic example of how a data science work requires engineering.

**Instructions:**
1. Code your own function containing the full workflow (train, predict, `print_quality_criterion`, display boxplots)

Test this function on various ML models, I have choosen various models from the litterature. **Of course, this work have to be performed by the Data scientist** (not enought time in this BE).

2. xgboost (see `XGBRegressor` on Google)
3. Gaussian Process (see `GaussianProcessRegressor` on Google)
4. *[More difficult]* Polynomial regression (Use the sklearn `PolynomialFeatures`). If it's too difficult, see answer in [this stack overflow post](https://stackoverflow.com/questions/54891965/multivariate-polynomial-regression-with-python).
5. *[Facultative]*: Test other ML approaches. You can find [here](https://scikit-learn.org/stable/supervised_learning.html) the regression models available on sklearn
6. *[Facultative]*: Test Multi-Layer Perceptron
    



### 4.1 Build your workflow function

### 4.2 XGBOOST

XGBoost is a technique of ML using trees. Read [this doc](https://medium.com/analytics-vidhya/introduction-to-xgboost-algorithm-d2e7fad76b04) for a gentle introduction, and [this one](https://xgboost.readthedocs.io/en/stable/tutorials/model.html) for a details. 

![img/xgboost.png](img/xgboost.png)

*Illusration of the XGBoost prediction:*

**XGBoost without scaling**

**XGBoost with scaling**

### 4.3/ Gaussian process

Find a very good introduction to Gaussian Process here: https://distill.pub/2019/visual-exploration-gaussian-processes/.

In a nutshell, GPs (Gaussian Processes) are a mixture of gaussian estimator. 

![img/gaussian_process.png](img/gaussian_process.png)

*Illustration of a Gaussian process regression*

**Gaussian Process without scaling**

**Gaussian Process with scaling**

### 4.4/ Polynomial regression

If you are blocked, see [this stackoverflow post](https://stackoverflow.com/questions/54891965/multivariate-polynomial-regression-with-python).

### 4.5/ Other models


Test other ML approaches. You can find [here](https://scikit-learn.org/stable/supervised_learning.html) the regression models available on sklearn.

**Random Forest**

**MultiOutputRegressor**

You may want to test models specialized in each feature (here, build 44 models, one per node). This (naive) approach is usefull to test sklearn models managing single outputs.

Detailes [here](https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputRegressor.html).

### 4.6/ Multi-Layer Perceptron

This is an introduction to MLP. If you are begining with Neural Network, we advise to use the [Keras](https://www.tensorflow.org/tutorials/keras/regression?hl=fr#regression_with_a_deep_neural_network_dnn) framework.

Keras is part of the Tensorflow project (Google): one of the main Neural Network library for Python. The other main Neural Network library is [pytorch](https://pytorch.org/). 

In [ ]:
# Imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


# 1/ Define your network
# ---------------------------

# Initiate the network in "sequential" mode
model = Sequential() 

# Layer 1
model.add(Dense(30, input_shape=(X_train.shape[1],)))  # input shape must be specified in layer 1
# X_train.shape[1] = 12 -> 12 parameters in input of the network

# Layer 2, 3, ... (one line per layer)
model.add(Dense(20))
model.add(Dense(5))
model.add(Dense(30))

# Last layer shall predict 45 Temperatures (`y_train.shape[1]`: nb of columns of the outputs (T°))
model.add(Dense(y_train.shape[1]))

# 2/ Compile model
# ---------------------------
# This command "compile" the model, creating the C++ graph of computation.
# Also permits to choose some hyper-parameters
model.compile(loss='mean_squared_error',  # Choose the cost function (here MSE)
              optimizer='adam',  # choose the optimization algorithm used (here ADAM: state of the art)
              )

# 3/ Display layering of your network
# ---------------------------
model.summary()

In [ ]:
# 4/ Train model
# ---------------------------
model.fit(X_train_scaled,
          y_train_scaled,
          epochs=60,  # Number of iteration of the training algorithm
          validation_data=(X_test_scaled, y_test_scaled),
          verbose=1  # verbosity of the logs: 0 for no message
         )

# 5/ Validate
# ---------------------------
y_pred_scaled = model.predict(X_test_scaled)
y_pred = scaler_out.inverse_transform(y_pred_scaled)

evaluate_predictions(y_pred)

# 5/ Execrice 3: Optimize your model

You have found some interesting models. But you have tested it in default configuration... Each models have several parameters (e.g. `n_estimators, max_depth, min_child_weight...` for XGBOOST). You can modify these parameters to see if it's increase (or decrease) the performance. This process is called **hyper-parameters tunning**. It is a mandatory step in the data scientist workflow.

**Purpose of the exercice:** Test tools to search the better set of hyper-parameters.

**Instructions:**
1. From an example of code, optimize manually the XGBOOST model proposed. 
2. Evaluate in a loop Polynom model from degree 2 to degree 5
3. Use the `RandomSearchCV` algorithm to automate the process
4. Use a state of the art algorithm (from `hyperopt` library) to select the better set of hyper-parameters

Find the best model !

### 5.1/ Tune manually an XGBoost model

Here are all the hyper-parameters of the XGBoost. Try to find manually the best set of hyper-parameters. Find [here](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBRegressor) the list of available hyper-parameters.

In [ ]:
xgbr = XGBRegressor(max_depth= 6,  # maximum depth of the trees estimators
                    gamma=0.1,  # min loss reduction required to create a new leaf
                    learning_rate=0.004,  # Learning rate of the optim. algorithm
                    n_estimators=200,  # Number of trees
                    subsample=0.7,  # % of the training 
                    colsample_bytree=0.7,  # % of cols used to construct each tree
                    verbosity=1  # do not touch this (level of logs)
                   )

# Test with and without scaling
train_evaluate(model=xgbr, scaled=True)

train_evaluate(model=xgbr, scaled=False)

### 5.2/ Test multiple polynom degree

On step toward automation: test all polynom from degree 2 to degree 5. 

### 5.3/ Automating Hyper-parameter search

**Basic approach**

You can automate the hyper-parameter tunning based on a very simple tool: `RandomSearchCV`. It consist in:

* A. Detail the list of parameters to tune (and their ranges of values)
* B. The algorithm will choose randomly set of hyper-parameters and evaluate the corresponding trained model
    * This action is done multiple times (`n_iter`)
    * A cross validation is performed to robustify the result 
    
![img/cv.png](img/cv.png)  
    
Some tips and tricks and doc [here](https://scikit-learn.org/stable/modules/grid_search.html#).

In [ ]:
# Number of test to launch
NB_TRIAL = 3

# List of parameters to tune and their ranges
params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'n_estimators': [100, 500, 1000],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
        }

Implement the tunning of hyper-parameters using the sklearn `RandomSearchCV` algorithm (google it). 

### 5.4/ Optimize hyper-parameters using `hyperopt`

You can use a dedicated library to tune your hyper-parameters. Algorithm available are based on optimization techniques. 

This [post](https://towardsdatascience.com/hyperopt-demystified-3e14006eb6fa) details a lot what is behind it. In a nutshell, it is an iterative research algorithm based Expected Improvement computation. 

From this piece of code, find the better hyper parameters of your best model.

In [ ]:
# Code taken from: https://www.kaggle.com/code/merrickolivier/hyperopt-and-xgbregressor-bayesian-optimization/notebook

NB_TRIAL = 3


space = {'learning_rate': hp.uniform('learning_rate', 0.001, 0.1),  # for a float value
         'max_depth': hp.randint('max_depth', 2, 7)  # for an integer value
        }

def hyperparameter_tuning(space):
    model = XGBRegressor(**space, verbosity=0)
    
    #Define evaluation datasets.
    evaluation = [(X_train, y_train), (X_test, y_test)]
    
    #Fit the model. Define evaluation sets, early_stopping_rounds, and eval_metric.
    model.fit(X_train,
              y_train,
              )

    #Obtain prediction and rmse score.
    pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, pred, squared=False)
    print ("SCORE:", rmse)
    
    #Specify what the loss is for each model.
    return {'loss':rmse, 'status': STATUS_OK, 'model': model}
         
         
#Run 20 trials.
trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=NB_TRIAL,
            trials=trials)

print(best)

In [ ]:
train_evaluate(XGBRegressor(**best), scaled=False)